We now implement our standard RG procedure for scalar fields. First generate samples, then use a suitable analog of block-spin and re-learn after several steps.

In [ ]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, JuMP, Ipopt, Plots

# Define the DualSchwingerModel2D structure for the 2D model
struct DualSchwingerModel2D
    lattice_size::Int  # Lattice size (NxN)
    λ::Float64         # Cosine interaction strength
    mu2::Float64       # Mass parameter squared (mu^2)
    θ::Float64         # Constant phase shift
end
#A function to help normalize the values of phi


# Function to calculate the total energy (kinetic + potential) of a 2D field configuration
function calculate_energy_2d(phi::Matrix{T}, model::DualSchwingerModel2D) where T<:Real
    nx, nt = size(phi)  # Dimensions of the lattice

    # Initialize energy terms using the type of `phi`
    kinetic_energy = zero(T)
    gradient_energy = zero(T)
    mass_energy = zero(T)
    cosine_energy = zero(T)

    # Loop through all lattice sites and calculate each term in the Hamiltonian
    for x in 1:nx
        for t in 1:nt
            # Calculate the temporal derivative term using discrete differences
            t_prev = mod1(t - 1, nt)  # Time neighbor with periodic boundary conditions
            kinetic_energy += T(0.5) * (phi[x, t] - phi[x, t_prev])^2

            # Calculate the spatial gradient term using discrete differences with `x_next` (right neighbor)
            x_prev = mod1(x - 1, nx)  # Spatial right neighbor with periodic boundary conditions
            gradient_energy += T(0.5) * (phi[x, t] - phi[x_prev, t])^2

            # Mass term using mu^2
            mass_energy += T(0.5) * model.mu2 * phi[x, t]^2

            # Cosine interaction term
            cosine_energy += -model.λ * cos((Float64(sqrt(4*π)) * phi[x, t]) - model.θ)
        end
    end

    # Return the total energy, scaled by χ
    total_energy = (kinetic_energy + gradient_energy + mass_energy + cosine_energy)
    return total_energy
end

# Define the DualSchwingerModelDensity2D structure for 2D
struct DualSchwingerModelDensity2D
    dim::Int
    model::DualSchwingerModel2D
end

# Implement the log-density function for DualSchwingerModelDensity2D
function LogDensityProblems.logdensity(p::DualSchwingerModelDensity2D, phi_vector::Vector{T}) where T<:Real
    # Reshape the vector back into a 2D matrix
    N = p.model.lattice_size
    phi = reshape(phi_vector, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::DualSchwingerModelDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{DualSchwingerModelDensity2D}) = LogDensityProblems.LogDensityOrder{2}()

# Function to generate data for the model using Hamiltonian Monte Carlo (HMC)
function generate_DSM_data_hmc_2d(model::DualSchwingerModel2D, n_samples::Int, n_adapts::Int = 3000)
    lattice_size = model.lattice_size
    phi4_target = DualSchwingerModelDensity2D(lattice_size^2, model)

    # Define the initial 2D field configuration (NxN matrix)
    initial_phi = randn(lattice_size, lattice_size)

    # Define a Hamiltonian system
    metric = DiagEuclideanMetric(lattice_size^2)
    hamiltonian = Hamiltonian(metric, phi4_target, ForwardDiff)

    # Define a leapfrog solver with an initial step size chosen heuristically
    initial_epsilon = find_good_stepsize(hamiltonian, vec(initial_phi))
    integrator = Leapfrog(initial_epsilon)

    # Define an HMC sampler with Multinomial sampling and Generalized No-U-Turn
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    # Run the sampler to draw samples from the model
    samples_vector, _ = sample(hamiltonian, kernel, vec(initial_phi), n_samples, adaptor, n_adapts; progress=false)

    # Reshape the samples back into 2D matrices
    samples = [reshape(sample, lattice_size, lattice_size) for sample in samples_vector]

    return samples
end

# Score matching objective function for 2D using mu^2
# Score matching objective function for 2D using mu^2
function score_matching_objective_2d(λ::T, mu2::T, θ::T, data::Vector{Matrix{Float64}}) where T<:Real
    nx, nt = size(data[1])
    n_samples = length(data)
    obj = zero(T)
    β =  Float64(sqrt(4*π))
    
    for sample in data
        for x in 1:nx
            for t in 1:nt
                phi_xt = sample[x, t]

                # Nearest neighbors with periodic boundary conditions
                t_prev = mod1(t - 1, nt)
                x_prev = mod1(x - 1, nx)
                t_next = mod1(t + 1, nt)
                x_next = mod1(x + 1, nx)
                
                # Sum of neighboring phi values
                sum_neighbor_phi = sample[x, t_prev] + sample[x, t_next] + sample[x_prev, t] + sample[x_next, t]

                # Correct score function s(phi_xt)
                score_xt = -((4 * phi_xt - sum_neighbor_phi) + mu2 * phi_xt + λ * β * sin(β * phi_xt - θ))

                # Correct second derivative s'(phi_xt)
                s_prime = -(4 + mu2 + λ * β^2 * cos(β * phi_xt - θ))

                # Add to the objective
                obj += T(0.5) * score_xt^2 + s_prime
            end
        end
    end

    # Normalize the objective by the number of samples
    return (obj / n_samples)
end

const RealMatrix = Matrix{Float64}
#Block-spin RG function for 2D lattice
function block_spin_rg(samples::Vector{RealMatrix})
    new_samples = Vector{RealMatrix}(undef, length(samples))
    for (i, φ) in enumerate(samples)
        N = size(φ,1)
        N2 = div(N,2)
        φ_new = zeros(Float64, N2, N2)
        for ix in 1:N2, it in 1:N2
            # average over 2x2 block
            block = φ[2ix-1:2ix, 2it-1:2it]
            φ_new[ix,it] = median(vec(block))
        end
        new_samples[i] = φ_new
    end
    return new_samples
end

# Function to estimate the parameters using score matching in 2D with mu^2
function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}})    
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, "print_level", 0)

    # Define optimization variables with mu2 instead of μ
    @variable(model, λ)
    @variable(model, mu2)
    @variable(model, 0.0 <= θ <= Float64(2*π))
    
    # Register the objective function using a wrapper that splats the inputs
    score_matching_wrapper = (λ, mu2, θ) -> score_matching_objective_2d(λ, mu2, θ, data)
    register(model, :score_matching_obj, 3, score_matching_wrapper; autodiff=true)

    # Set the nonlinear objective
    @NLobjective(model, Min, score_matching_obj(λ, mu2, θ))
    
    # Run the optimization
    optimize!(model)
    println("Final objective value: ", objective_value(model))
    println("Optimizer termination status: ", termination_status(model))

    if !(termination_status(model) in [MOI.OPTIMAL, MOI.LOCALLY_SOLVED])
        @warn "Optimization did not converge to an optimal solution"
    end
    
    # Retrieve optimized parameter values
    estimated_lambda = value(λ)
    estimated_mu2 = value(mu2)
    estimated_theta = value(θ)
    
    return estimated_lambda, estimated_mu2, estimated_theta
end

lattice_size = 32
n_samples = 15000
true_model = DualSchwingerModel2D(lattice_size, 0.8, 0.4, pi)  # Use mu^2 for true_model

samples = generate_DSM_data_hmc_2d(true_model, n_samples)  # HMC sampling
        
# Estimate parameters using score matching
λ_est, mu2_est, theta_est = estimate_parameters_score_matching_2d(samples)
println("Estimated parameters: λ=$λ_est, mu^2=$mu2_est, θ=$theta_est")
